In [ ]:
# (planta, [personas], [ascensores])
# ascensores_lentos = 0,1,2,3  ascensor_rapido = 4
estado_ini = {
    (0, [], [0,4])
    (1, [2,4], []),
    (2, [0], []),
    (3, [], []),
    (4, [1], [1,2]),
    (5, [], []),
    (6, [], []),
    (7, [], []),
    (8, [3], [3]),
    (9, [], []),
    (10, [], []),
    (11, [], []),
    (12, [], [])
}

estado_fin = {}
    (0, [], [])
    (1, [3], []),
    (2, [], []),
    (3, [0], []),
    (4, [], []),
    (5, [], []),
    (6, [], []),
    (7, [], []),
    (8, [], []),
    (9, [4], []),
    (10, [], []),
    (11, [1], []),
    (12, [2], [])
}

'''self._actions = [('a',(0,0)), ('a',(0,1)), ('a',(0,2)), ('a',(0,3)), ('a',(0,4)),
                         ('a',(1,4)), ('a',(1,5)), ('a',(1,6)), ('a',(1,7)), ('a',(1,8)),
                         ('a',(2,4)), ('a',(2,5)), ('a',(2,6)), ('a',(2,7)), ('a',(2,8)),
                         ('a',(3,8)), ('a',(3,9)), ('a',(3,10)), ('a',(3,11)), ('a',(3,12)),
                         ('a',(4,0)), ('a',(4,2)), ('a',(4,4)), ('a',(4,6)), ('a',(4,8)), ('a',(4,10)), ('a',(4,12)),
                         ('p',(0,0)), ('p',(0,1)), ('p',(0,2)), ('p',(0,3)), ('p',(0,4)),
                         ('p',(1,0)), ('p',(1,1)), ('p',(1,2)), ('p',(1,3)), ('p',(1,4)),
                         ('p',(2,0)), ('p',(2,1)), ('p',(2,2)), ('p',(2,3)), ('p',(2,4)),
                         ('p',(3,0)), ('p',(3,1)), ('p',(3,2)), ('p',(3,3)), ('p',(3,4)),
                         ('p',(4,0)), ('p',(4,1)), ('p',(4,2)), ('p',(4,3)), ('p',(4,4)),
                        ]'''

In [2]:
# Suponemos que solo cabe una persona por ascensor
# ascensores_lentos = 0,1,2,3  ascensor_rapido = 4
estado_ini = {
    personas = [2, 4, 1, 8, 1]
    ascensores = [0, 4, 4, 8, 0]
}

estado_fin = {
    personas = [3, 11, 12, 1, 9]
    ascensores = []
}

SyntaxError: invalid syntax (<ipython-input-2-a819662b46dd>, line 4)

In [ ]:
# importamos las cosas que vamos a usar de aima
from search import *

class ProblemaAscensores(Problem):
    ''' Clase problema (formalizacion de nuestro problema) siguiendo la
        estructura que aima espera que tengan los problemas.'''
    def __init__(self, initial, goal=None):
        '''Inicializacion de nuestro problema.'''
        Problem.__init__(self, initial, goal)
        self._actions = [('a',(0,0)), ('a',(0,1)), ('a',(0,2)), ('a',(0,3)), ('a',(0,4)),
                         ('a',(1,4)), ('a',(1,5)), ('a',(1,6)), ('a',(1,7)), ('a',(1,8)),
                         ('a',(3,8)), ('a',(3,9)), ('a',(3,10)), ('a',(3,11)), ('a',(3,12)),
                         ('a',(4,0)), ('a',(4,2)), ('a',(4,4)), ('a',(4,6)), ('a',(4,8)), ('a',(4,10)), ('a',(4,12)),
                         ('p',(0,0)), ('p',(0,1)), ('p',(0,2)), ('p',(0,3)), ('p',(0,4)),
                         ('p',(1,0)), ('p',(1,1)), ('p',(1,2)), ('p',(1,3)), ('p',(1,4)),
                         ('p',(2,0)), ('p',(2,1)), ('p',(2,2)), ('p',(2,3)), ('p',(2,4)),
                        ]

    def actions(self, s):
        '''Devuelve las acciones validas para un estado.'''
        # las acciones validas para un estado son aquellas que al aplicarse
        # nos dejan en otro estado valido
        return [a for a in self._actions if self._is_valid(s, a)]

    def _is_valid(self, a):
        '''Determina si un estado es valido o no.'''
        if a[0] = 'p':
            x = a[1]
            if(self.personas[x[0]] == self.ascensores)
        return (s[0] >= s[1] or s[0] == 0) and ((3 - s[0]) >= (3 - s[1]) or s[0] == 3) and (0 <= s[0] <= 3) and (0 <= s[1] <= 3)

    def result(self, s, a):
        '''Devuelve el estado resultante de aplicar una accion a un estado
           determinado.'''
        # el estado resultante tiene la canoa en el lado opuesto, y con las
        # cantidades de misioneros y canibales actualizadas segun la cantidad
        # que viajaron en la canoa
        if s[2] == 0:
            return (s[0] - a[1][0], s[1] - a[1][1], 1)
        else:
            return (s[0] + a[1][0], s[1] + a[1][1], 0)

